# Day 2 - Lab 1: AI-Generated System Design & Database Seeding

**Objective:** Use the PRD artifact from Day 1 to generate a detailed SQL database schema, create realistic seed data, and then use those outputs to create and seed a live, local database file.

**Estimated Time:** 150 minutes

**Introduction:**
Welcome to Day 2! Today, we transition from *what* we're building to *how* we'll build it. In this lab, you will act as the lead architect for the Onboarding Tool. Your task is to use the PRD to define the data structure of the application and create a tangible database artifact that will be used for the rest of the course.

For definitions of key terms used in this lab, please refer to the [GLOSSARY.md](../../GLOSSARY.md).

## Step 1: Setup

We will load the `day1_prd.md` artifact from Day 1. This document is the primary source of truth for our project and provides the necessary context for the LLM to make intelligent design suggestions.

**Model Selection:**
Feel free to experiment with different models by changing the `model_name` in `setup_llm_client()`. Models with strong reasoning capabilities, like `gpt-4o`, `o3`, or `gemini-2.5-pro`, are excellent choices for design tasks.

**Helper Functions Used:**
- `setup_llm_client()`: To configure the API client.
- `get_completion()`: To send prompts to the LLM.
- `load_artifact()`: To read the PRD from the `artifacts` directory.
- `save_artifact()`: To save the generated SQL schema and seed data.
- `clean_llm_output()`: To remove markdown fences from the generated SQL code.

In [1]:
import sys
import os
import sqlite3

# Add the project's root directory to the Python path to ensure 'utils' can be imported.
try:
    project_root = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
except IndexError:
    project_root = os.path.abspath(os.path.join(os.getcwd()))

if project_root not in sys.path:
    sys.path.insert(0, project_root)

from utils import setup_llm_client, get_completion, save_artifact, load_artifact, clean_llm_output, prompt_enhancer

client, model_name, api_provider = setup_llm_client(model_name="gemini-2.5-pro")

# Load the PRD from Day 1
prd_content = load_artifact("artifacts/day1_prd.md")
if not prd_content:
    print("Warning: Could not load day1_prd.md. Lab may not function correctly.")

2025-10-29 12:52:01,288 ag_aisoftdev.utils INFO LLM Client configured provider=google model=gemini-2.5-pro latency_ms=None artifacts_path=None


## Step 2: The Challenges

### Challenge 1 (Foundational): Generating the SQL Schema

**Task:** Use the PRD to generate a normalized SQL schema for the application.

**Instructions:**
1.  Create a prompt that instructs the LLM to act as a Database Administrator (DBA).
2.  Provide the `prd_content` as context.
3.  Ask the LLM to design a normalized SQL schema with at least two tables (e.g., `users` and `onboarding_tasks`).
4.  The output should be the raw `CREATE TABLE` statements.
5.  Save the generated SQL to `artifacts/schema.sql`.

In [2]:
# TODO: Write a prompt to generate the SQL schema from the PRD.
schema_prompt = f"""
You are a database administrator and SQL expert. Based on the following Product Requirements Document (PRD), generate a complete SQL schema for a relational database. Ensure that the schema includes tables, columns with appropriate data types, primary keys, foreign keys, and any necessary constraints.
PRD:
{prd_content}
The output should be the raw CREATE TABLE statements without any additional explanations or comments.
Ensure the schema is simple yet comprehensive enough to cover all aspects mentioned in the PRD.
Ensure the schema is syntactically correct for SQLite.
Drop any tables if they already exist before creating them.
Ensure proper relationships between tables using foreign keys.
"""
enhanced_schema_prompt = prompt_enhancer(schema_prompt)
print("--- SQL Enhanced Schema Generation Prompt ---")
print(enhanced_schema_prompt)

print("--- Generating SQL Schema ---")
if prd_content:
    generated_schema = get_completion(
        enhanced_schema_prompt,
        client,
        model_name,
        api_provider,
        temperature=0.3,
    )
    
    # Clean up the generated schema using our helper function
    cleaned_schema = clean_llm_output(generated_schema, language='sql')
    print(cleaned_schema)
    
    # Save the cleaned schema
    save_artifact(cleaned_schema, 'artifacts/schema.sql', overwrite=True)
else:
    print("Skipping schema generation because PRD is missing.")
    cleaned_schema = ""

2025-10-29 12:52:01,854 ag_aisoftdev.utils INFO LLM Client configured provider=openai model=o3 latency_ms=None artifacts_path=None


--- SQL Enhanced Schema Generation Prompt ---
<persona>
You are a Senior Database Architect and SQL expert.
</persona>

<context>
Below is the complete Product Requirements Document (PRD) for “OnboardPro,” an employee-onboarding platform. Use it as the single source of truth for understanding the data domain, entities, and relationships.

[START_PRD]
# Product Requirements Document: OnboardPro
| Status | Draft |
| Author | Product Team – Onboarding |
| Version | 1.0 |
| Last Updated | 2023-10-27 |
…(full PRD exactly as provided by the user, unchanged)…
[END_PRD]
</context>

<instructions>
1. Think step by step to identify all entities, their attributes, and relationships needed to satisfy every functional and non-functional requirement in the PRD. Perform this reasoning silently and do NOT reveal it in the final answer.

2. Generate a complete, syntactically correct SQLite schema that:
   • Creates every necessary table with appropriate column names and data types.  
   • Defines prima

### Challenge 2 (Intermediate): Generating Realistic Seed Data

**Task:** Prompt the LLM to generate realistic seed data that conforms to the schema you just created.

**Instructions:**
1.  Create a new prompt.
2.  Provide both the `prd_content` and the `cleaned_schema` as context.
3.  Instruct the LLM to generate 5-10 realistic `INSERT` statements for your tables.
4.  The data should be relevant to a new hire onboarding tool (e.g., sample user names and task titles like "Complete HR Paperwork").
5.  Save the generated `INSERT` statements to `artifacts/seed_data.sql`.

In [3]:
# TODO: Write a prompt to generate realistic seed data.
seed_data_prompt = f"""
Using the following SQL schema and Product Requirements Document, generate realistic SQL INSERT statements to seed the database with sample data.
SQL Schema:
{cleaned_schema}
Product Requirements Document (PRD):
{prd_content}
Author high-quality data that:
- Populates every table that contains NOT NULL or foreign key constraints (roles, departments, teams, users, templates, template_tasks, onboarding_plans, onboarding_tasks, documents, task_documents, notifications, surveys, survey_questions, survey_assignments, survey_responses).
- Uses only column names that exist in the schema and provides values for every required column.
- Respects ENUM/CHECK constraints (e.g., status values, notification types, question_type options) and uses ISO-8601 timestamps or CURRENT_TIMESTAMP where appropriate.
- Keeps identifiers consistent across inserts (e.g., onboarding_tasks.assignee_user_id must reference an existing users.user_id).
Return 10-15 INSERT statements with no markdown code fences or commentary.
"""
enhanced_data_prompt = prompt_enhancer(seed_data_prompt)
print("--- SQL Enhanced Seed Data Generation Prompt ---")
print (enhanced_data_prompt)

print("--- Generating Seed Data ---")
if prd_content and cleaned_schema:
    generated_seed_data = get_completion(
        enhanced_data_prompt,
        client,
        model_name,
        api_provider,
        temperature=0.2,
    )
    
    # Clean up the generated seed data
    cleaned_seed_data = clean_llm_output(generated_seed_data, language='sql')
    print(cleaned_seed_data)
    
    # Save the cleaned seed data
    save_artifact(cleaned_seed_data, 'artifacts/seed_data.sql', overwrite=True)
else:
    print("Skipping seed data generation because PRD or schema is missing.")

2025-10-29 12:52:57,748 ag_aisoftdev.utils INFO LLM Client configured provider=openai model=o3 latency_ms=None artifacts_path=None


--- SQL Enhanced Seed Data Generation Prompt ---
<persona>
You are a senior SQL database architect and HR-tech domain expert who routinely seeds relational databases with realistic, constraint-compliant sample data.
</persona>

<context>
You are preparing seed data for a fresh installation of the OnboardPro platform.  
The full SQL schema (already executed) contains these tables and constraints:

• companies(company_id PK, name UNIQUE NOT NULL, created_at, updated_at)  
• users(user_id PK, company_id FK, email UNIQUE, password_hash, first_name, last_name, role CHECK('Admin','Hiring Manager','New Hire'), manager_id FK users.user_id, timestamps)  
• documents(document_id PK, company_id FK, file_name, file_path UNIQUE, file_type, uploaded_by_user_id FK, timestamps)  
• onboarding_plans(plan_id PK, company_id FK, title UNIQUE per company, description, created_by_user_id FK, is_active, timestamps)  
• tasks(task_id PK, plan_id FK, title, description, due_days_after_start, order_in_plan, doc

### Challenge 3 (Advanced): Creating and Seeding a Live Database

**Task:** This is a critical technical step. You will write a Python script to execute the generated SQL files, creating a live `onboarding.db` file that your application will use.

**Instructions:**
1.  Complete the `create_database` function below.
2.  The function should first connect to (and thus create) a SQLite database file named `artifacts/onboarding.db`.
3.  It should then open and execute the `schema.sql` file to create the tables.
4.  Finally, it should open and execute the `seed_data.sql` file to populate the tables.
5.  Use a `try...finally` block to ensure the database connection is always closed, even if an error occurs.

> **Hint:** The `try...finally` block is a crucial Python pattern. The code in the `finally` block will run whether the `try` block succeeds or fails, making it the perfect place to ensure resources like database connections are always closed.

**Expected Quality:** A physical `onboarding.db` file in your `artifacts` folder. This is a tangible asset that proves your design is valid and provides a concrete foundation for backend development.

In [4]:
def create_database(db_path, schema_path, seed_path):
    """Creates and seeds a SQLite database from SQL files."""
    if not os.path.exists(schema_path):
        print(f"Error: Schema file not found at {schema_path}")
        return
    
    conn = None
    try:
        # TODO: Connect to the SQLite database. This will create the file if it doesn't exist.
        conn = sqlite3.connect(db_path)
        cursor = conn.cursor()
        print(f"Successfully connected to database at {db_path}")
        
        # Read the content of the schema file using load_artifact.
        schema_sql = load_artifact(os.path.relpath(schema_path, project_root))
        if not schema_sql:
            print("Error: Schema SQL is empty or missing.")
            return
        
        # Execute the schema SQL script.
        cursor.executescript(schema_sql)
        print("Tables created successfully.")
        
        # Check if the seed data file exists. If it does, load and execute it.
        if os.path.exists(seed_path):
            seed_sql = load_artifact(os.path.relpath(seed_path, project_root))
            if seed_sql:
                cursor.executescript(seed_sql)
                print("Seed data inserted successfully.")
            else:
                print("Warning: Seed SQL is empty. Skipping data load.")
        else:
            print("Seed data file not found; skipping data load.")
        
        # Commit the changes to the database.
        conn.commit()
    except sqlite3.Error as e:
        print(f"Database error: {e}")
    finally:
        # Ensure the connection is closed if it was opened.
        if conn:
            conn.close()
            print("Database connection closed.")

# Define file paths
db_file = os.path.join(project_root, "artifacts", "onboarding.db")
schema_file = os.path.join(project_root, "artifacts", "schema.sql")
seed_file = os.path.join(project_root, "artifacts", "seed_data.sql")

# Execute the function
create_database(db_file, schema_file, seed_file)

Successfully connected to database at c:\Workspace\AG-AISOFTDEV\artifacts\onboarding.db
Tables created successfully.
Seed data inserted successfully.
Database connection closed.


## Lab Conclusion

Excellent work! You have now moved from abstract requirements to a concrete, physical database artifact. You've used an LLM to design a schema, generate realistic test data, and then used a Python script to bring that database to life. This `onboarding.db` file is the foundation upon which we will build our API in Day 3.

> **Key Takeaway:** The ability to generate structured data definitions (like a SQL schema) from unstructured text (like a PRD) is a core skill in AI-assisted development. It automates a critical and often time-consuming design step.